In [2]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

# 2.4. Обучение и проверка моделей
Обучите Vowpal Wabbit на выборке stackoverflow_train.vw 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3). Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.
Проверяйте долю правильных ответов на выборке stackoverflow_valid.vw. Выберите лучшую модель и проверьте качество на выборке stackoverflow_test.vw.

 -oaa 10 – указываем, что классификация на 10 классов

-d – путь к данным
-f – путь к модели, которая будет построена
-b 28 – используем 28 бит для хэширования, то есть признаковое пространство ограничено 228 признаками, что в данном случае больше, чем число уникальных слов в выборке (но потом появятся би- и триграммы, и ограничение размерности признакового пространства начнет работать)
также указываем random seed
--quiet

In [ ]:
from tqdm import tqdm
pbar = tqdm()

best_score={}
def validation(text):#валидация
    pbar.update(1)
    !vw -t -i /ml/vw_model1_10mln.vw -d /ml/stackoverflow_valid.vw -p /ml/vw_valid_10mln_pred1.csv --random_seed 17 --quiet
    #оценка
    vw_pred = np.loadtxt('/ml/vw_valid_10mln_pred1.csv')
    test_labels = np.loadtxt('/ml/stackoverflow_valid_labels.txt')
    best_score[text]=accuracy_score(test_labels, vw_pred)
    print('accuracy_score ',text,': ',best_score[text])


#обучение
!vw --passes 1 --ngram 1 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 1 --ngram 1')

!vw --passes 3 --ngram 1 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 3 --ngram 1')

!vw --passes 5 --ngram 1 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 5 --ngram 1')


!vw --passes 1 --ngram 2 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 1 --ngram 2')

!vw --passes 3 --ngram 2 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 3 --ngram 2')

!vw --passes 5 --ngram 2 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 5 --ngram 2')


!vw --passes 1 --ngram 3 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 1 --ngram 3')

!vw --passes 3 --ngram 3 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 3 --ngram 3')

!vw --passes 5 --ngram 3 --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw --bit_precision 28 --random_seed 17 -c --quiet
validation('--passes 5 --ngram 3')


pbar.close()
sv=sorted(best_score, key=best_score.get, reverse=True)
print(sv[0],':',best_score[sv[0]])